In [1]:
# Data columns (total 29 columns):
#  #   Column                  Non-Null Count  Dtype  
# ---  ------                  --------------  -----  
#  0   ID                      499 non-null    int64  
#  1   Resort                  499 non-null    object 
#  2   Latitude                499 non-null    float64
#  3   Longitude               499 non-null    float64
#  4   Country                 499 non-null    object 
#  5   Continent               499 non-null    object 
#  6   Price                   499 non-null    int64  
#  7   Season                  499 non-null    object 
#  8   Highest point           499 non-null    int64  
#  9   Lowest point            499 non-null    int64  
#  10  Beginner slopes         499 non-null    int64  
#  11  Intermediate slopes     499 non-null    int64  
#  12  Difficult slopes        499 non-null    int64  
#  13  Total slopes            499 non-null    int64  
#  14  Longest run             499 non-null    int64  
#  15  Snow cannons            499 non-null    int64  
#  16  Surface lifts           499 non-null    int64  
#  17  Chair lifts             499 non-null    int64  
#  18  Gondola lifts           499 non-null    int64  
#  19  Total lifts             499 non-null    int64  
# ...
#  27  country_slope_rank      499 non-null    float64
#  28  country_cannon_rank     499 non-null    float64
# dtypes: float64(6), int64(15), object(8)

In [16]:
# import packages

from dash import Dash, dcc, html, dash_table
import dash_bootstrap_components as dbc
from dash.dependencies import Output, Input
from dash.exceptions import PreventUpdate
from dash_bootstrap_templates import load_figure_template

import plotly.express as px
import pandas as pd
import numpy as np

# import data and assign a rank to certain metrics

resorts = (
    pd.read_csv("data/resorts.csv", encoding = "ISO-8859-1")
    .assign(
        country_elevation_rank = lambda x: x.groupby("Country", as_index=False)["Highest point"].rank(ascending=False),
        country_price_rank = lambda x: x.groupby("Country", as_index=False)["Price"].rank(ascending=False),
        country_slope_rank = lambda x: x.groupby("Country", as_index=False)["Total slopes"].rank(ascending=False),
        country_cannon_rank = lambda x: x.groupby("Country", as_index=False)["Snow cannons"].rank(ascending=False),
    ))

# application

app = Dash(__name__)

app.layout = html.Div([
    dcc.Slider(id = "maxprice",
               min = 0,
               max = 140,
               step = 20,
               value = 80
               ),
    dcc.Checklist(id = "checklist",
                 options = ["Has summer skiing", 
                            "Has night skiing", 
                            "Has snow parks"],
                 value = []),
    dcc.Graph(id = "map")
])

@app.callback(
    Output("map", "figure"),
    Input("maxprice", "value")
)

def update_dash(pricelimit):

    df = resorts.query("Price <= @pricelimit")

    figure = px.density_mapbox(
        df,
        lat = "Latitude",
        lon = "Longitude",
        z = "Total slopes",
        mapbox_style = "open-street-map",
        zoom = 2.5,
        center = {"lat": 52.5, "lon": 15},
        height = 600,
        width = 800,
        radius = 15,
        title = "Slope density in resorts",
        hover_name="Resort")

    return figure

if __name__ == "__main__":
    app.run_server(debug = True)